In [9]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import tensorflowjs as tfjs

# Read the data from CSV file
try:
    data = pd.read_csv("ICD10_openmed_UTF8.csv", delimiter=";", encoding="utf-8")
except FileNotFoundError:
    print("Error: CSV file not found. Please provide the correct path.")
    exit()
except pd.errors.ParserError:
    print("Error: CSV parsing error. Check the format of your data.")
    exit()

# Tokenisierung und Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['term']) 
sequences = tokenizer.texts_to_sequences(data['term'])
max_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_length)
y = data["icd10"]

# Convert labels to numeric indices
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training, test, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Überprüfen Sie die Form von X
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_val: {X_val.shape}")
print(f"Shape of y_val: {y_val.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")

# Überprüfen Sie, ob X ein Numpy-Array ist
if not isinstance(X, np.ndarray):
    X = np.array(X)

# Create the model architecture
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))
model.add(LSTM(100))
model.add(Dense(len(set(y)), activation='softmax'))  # Use the number of unique labels as the output dimension

# Zusammenfassung des Modells
model.summary()

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Save the model
model.save('nlp_model.h5')

# Convert the model to TensorFlow.js format
tfjs.converters.save_keras_model(model, 'tfjs_model')


Shape of X_train: (39232, 18)
Shape of y_train: (39232,)
Shape of X_val: (8407, 18)
Shape of y_val: (8407,)
Shape of X_test: (8408, 18)
Shape of y_test: (8408,)


/Users/tizander/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - accuracy: 0.1091 - loss: 6.2671 - val_accuracy: 0.2129 - val_loss: 5.3327
Epoch 2/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.2406 - loss: 4.9893 - val_accuracy: 0.3278 - val_loss: 4.5448
Epoch 3/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.3584 - loss: 4.0997 - val_accuracy: 0.4263 - val_loss: 3.9012
Epoch 4/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.4630 - loss: 3.3617 - val_accuracy: 0.4876 - val_loss: 3.4509
Epoch 5/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - accuracy: 0.5197 - loss: 2.8588 - val_accuracy: 0.5248 - val_loss: 3.1315
Epoch 6/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.5705 - loss: 2.4446 - val_accuracy: 0.5572 - val_loss: 2.9249
Epoch 7/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.6074 - loss: 2.1291 - val_accuracy: 0.5773 - val_loss: 2.7842
Epoch 8/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 22s 36ms/step - accuracy: 0.6407 - loss: 1.8617 - 

Test Loss: 2.594663619995117
Test Accuracy: 0.6173881888389587


failed to lookup keras version from the file,
    this is likely a weight only file


Exception: Error dumping weights, duplicate weight name kernel

In [11]:
## RNN MODEL

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import tensorflowjs as tfjs

# Read the data from CSV file
try:
    data = pd.read_csv("ICD10_openmed_UTF8.csv", delimiter=";", encoding="utf-8")
except FileNotFoundError:
    print("Error: CSV file not found. Please provide the correct path.")
    exit()
except pd.errors.ParserError:
    print("Error: CSV parsing error. Check the format of your data.")
    exit()

# Tokenisierung und Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['term']) 
sequences = tokenizer.texts_to_sequences(data['term'])
max_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_length)

# Konvertieren der Labels zu numerischen Indizes
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["icd10"])

# Split the data into training, test, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Überprüfen Sie die Formen der Daten
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_val: {X_val.shape}")
print(f"Shape of y_val: {y_val.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")

# Create the RNN model architecture
model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))
model_rnn.add(LSTM(100))
model_rnn.add(Dense(len(set(y)), activation='softmax'))

# Zusammenfassung des Modells
model_rnn.summary()

# Compile the model
model_rnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model_rnn.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

# Evaluate the model on test data
test_loss, test_accuracy = model_rnn.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Save the model
model_rnn.save('nlp_rnn_model.h5')

# Convert the model to TensorFlow.js format
tfjs.converters.save_keras_model(model_rnn, 'tfjs_rnn_model')


Shape of X_train: (39232, 18)
Shape of y_train: (39232,)
Shape of X_val: (8407, 18)
Shape of y_val: (8407,)
Shape of X_test: (8408, 18)
Shape of y_test: (8408,)


/Users/tizander/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 18s 27ms/step - accuracy: 0.1074 - loss: 6.2751 - val_accuracy: 0.1632 - val_loss: 5.4198
Epoch 2/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.2099 - loss: 5.1106 - val_accuracy: 0.2796 - val_loss: 4.8019
Epoch 3/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.3180 - loss: 4.3599 - val_accuracy: 0.3848 - val_loss: 4.1608
Epoch 4/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 18s 29ms/step - accuracy: 0.4166 - loss: 3.6603 - val_accuracy: 0.4589 - val_loss: 3.6696
Epoch 5/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.4906 - loss: 3.0817 - val_accuracy: 0.5068 - val_loss: 3.3034
Epoch 6/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 22s 36ms/step - accuracy: 0.5468 - loss: 2.6320 - val_accuracy: 0.5399 - val_loss: 3.0465
Epoch 7/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - accuracy: 0.5921 - loss: 2.2671 - val_accuracy: 0.5638 - val_loss: 2.8798
Epoch 8/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.6251 - loss: 1.9716 - 

Test Loss: 2.6453397274017334
Test Accuracy: 0.615247368812561


failed to lookup keras version from the file,
    this is likely a weight only file


Exception: Error dumping weights, duplicate weight name kernel

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import tensorflowjs as tfjs
import nltk

# Download NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Read the data from CSV file
try:
    data = pd.read_csv("ICD10_openmed_UTF8.csv", delimiter=";", encoding="utf-8")
except FileNotFoundError:
    print("Error: CSV file not found. Please provide the correct path.")
    exit()
except pd.errors.ParserError:
    print("Error: CSV parsing error. Check the format of your data.")
    exit()

# Function for text preprocessing
def preprocess_text(text):
    # Tokenize
    tokens = word_tokenize(text)
    
    # Convert to lower case
    tokens = [token.lower() for token in tokens]
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming/Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

# Apply preprocessing to the text data
data['processed_term'] = data['term'].apply(preprocess_text)

# Tokenization and padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['processed_term']) 
sequences = tokenizer.texts_to_sequences(data['processed_term'])
max_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_length)

# Convert labels to numeric indices
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["icd10"])

# Split the data into training, test, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Check the shapes of the data
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_val: {X_val.shape}")
print(f"Shape of y_val: {y_val.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")
# Create the RNN model architecture
model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))
model_rnn.add(LSTM(100))
model_rnn.add(Dense(len(set(y)), activation='softmax'))

# Model summary
model_rnn.summary()

# Compile the model
model_rnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model_rnn.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

# Evaluate the model on test data
test_loss, test_accuracy = model_rnn.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Save the model
model_rnn.save('nlp_rnn_model.h5')

# Convert the model to TensorFlow.js format
tfjs.converters.save_keras_model(model_rnn, 'tfjs_rnn_model')


[nltk_data] Downloading package punkt to /Users/tizander/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tizander/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tizander/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Shape of X_train: (39232, 17)
Shape of y_train: (39232,)
Shape of X_val: (8407, 17)
Shape of y_val: (8407,)
Shape of X_test: (8408, 17)
Shape of y_test: (8408,)


/Users/tizander/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - accuracy: 0.1105 - loss: 6.2622 - val_accuracy: 0.2034 - val_loss: 5.3099
Epoch 2/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.2322 - loss: 4.9566 - val_accuracy: 0.3165 - val_loss: 4.5590
Epoch 3/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 18s 30ms/step - accuracy: 0.3491 - loss: 4.1146 - val_accuracy: 0.4195 - val_loss: 3.9361
Epoch 4/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - accuracy: 0.4458 - loss: 3.4338 - val_accuracy: 0.4733 - val_loss: 3.5183
Epoch 5/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.5034 - loss: 2.9425 - val_accuracy: 0.5123 - val_loss: 3.2438
Epoch 6/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 22s 37ms/step - accuracy: 0.5570 - loss: 2.5338 - val_accuracy: 0.5400 - val_loss: 3.0558
Epoch 7/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 18s 29ms/step - accuracy: 0.5840 - loss: 2.2439 - val_accuracy: 0.5648 - val_loss: 2.9014
Epoch 8/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.6208 - loss: 1.9608 - 

Test Loss: 2.669365882873535
Test Accuracy: 0.6110846996307373


failed to lookup keras version from the file,
    this is likely a weight only file


Exception: Error dumping weights, duplicate weight name kernel

In [13]:
## BERT NLP Model 

from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load the BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenisieren und Padding der Daten mit BERT-Tokenizer
def bert_encode(texts, tokenizer, max_len):
    input_ids = []
    attention_masks = []
    
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='tf',
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return np.array(input_ids), np.array(attention_masks)

X_input_ids, X_attention_masks = bert_encode(data['term'], bert_tokenizer, max_length=128)

# Split the data into training, test, and validation sets
X_train_ids, X_temp_ids, y_train, y_temp = train_test_split(X_input_ids, y, test_size=0.3, random_state=42)
X_val_ids, X_test_ids, y_val, y_test = train_test_split(X_temp_ids, y_temp, test_size=0.5, random_state=42)

X_train_masks, X_temp_masks, _, _ = train_test_split(X_attention_masks, y, test_size=0.3, random_state=42)
X_val_masks, X_test_masks, _, _ = train_test_split(X_temp_masks, y_temp, test_size=0.5, random_state=42)

# Create the BERT model architecture
model_bert = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(y)))

# Compile the model
optimizer = Adam(learning_rate=3e-5, epsilon=1e-8)
model_bert.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_bert.fit([X_train_ids, X_train_masks], y_train, validation_data=([X_val_ids, X_val_masks], y_val), epochs=3, batch_size=8)

# Evaluate the model on test data
test_loss, test_accuracy = model_bert.evaluate([X_test_ids, X_test_masks], y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Save the model
model_bert.save_pretrained('nlp_bert_model')
tfjs.converters.save_keras_model(model_bert, 'tfjs_bert_model')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/Users/tizander/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

TypeError: bert_encode() got an unexpected keyword argument 'max_length'